In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import socket

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Thresholds 
STILL_THRESHOLD = 8.0 
AGGRESSIVE_ENTER_THRESHOLD = 720.0 
AGGRESSIVE_EXIT_THRESHOLD = 650.0 
JERK_THRESHOLDS = np.array([1000.0] * 13) 

# landmarks
SELECTED_LANDMARKS = [0, 15, 16, 11, 12, 23, 24, 25, 26, 27, 28, 31, 32]

# Initialize EMA
smoothed_landmarks = np.zeros((len(SELECTED_LANDMARKS), 3))
ALPHA = 0.5 

# Parameters of Hystiris
SUSTAINED_FRAMES = 5
STILL_FRAMES = 10
state_counters = [0] * len(SELECTED_LANDMARKS)
still_counters = [0] * len(SELECTED_LANDMARKS)
current_states = ["Still"] * len(SELECTED_LANDMARKS)

def apply_ema(new_landmarks, smoothed_landmarks, alpha):
    return alpha * new_landmarks + (1 - alpha) * smoothed_landmarks

# Set up UDP socket
UDP_IP = "127.0.0.1" 
UDP_PORT = 7400  
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

def send_to_max(state):
    try:
        sock.sendto(state.encode(), (UDP_IP, UDP_PORT))
    except Exception as e:
        print(f"Error sending data to Max MSP: {e}")

def start_detection():
    global smoothed_landmarks

    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        previous_velocities = None
        previous_accelerations = None
        previous_time = None

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark])
                selected_landmarks = landmarks[SELECTED_LANDMARKS]
                smoothed_landmarks = apply_ema(selected_landmarks, smoothed_landmarks, ALPHA)

                current_time = time.time()
                if previous_time is not None:
                    delta_time = max(0.001, current_time - previous_time)
                    velocities = (smoothed_landmarks - selected_landmarks) / delta_time

                    if previous_velocities is not None:
                        accelerations = (velocities - previous_velocities) / delta_time

                        if previous_accelerations is not None:
                            jerks = (accelerations - previous_accelerations) / delta_time
                            acceleration_magnitudes = np.linalg.norm(accelerations, axis=1)
                            jerk_magnitudes = np.linalg.norm(jerks, axis=1)

                            for i, (acc_mag, jerk_mag) in enumerate(zip(acceleration_magnitudes, jerk_magnitudes)):
                                if acc_mag < STILL_THRESHOLD:
                                    still_counters[i] += 1
                                    if still_counters[i] >= STILL_FRAMES and current_states[i] != "Still":
                                        current_states[i] = "Still" 
                                        state_counters[i] = 0
                                else:
                                    still_counters[i] = 0

                                    if acc_mag > AGGRESSIVE_ENTER_THRESHOLD or jerk_mag > JERK_THRESHOLDS[i]:
                                        state_counters[i] += 1
                                        if state_counters[i] >= SUSTAINED_FRAMES and current_states[i] != "Aggressive":
                                            current_states[i] = "Aggressive"
                                            state_counters[i] = 0
                                    elif acc_mag < AGGRESSIVE_EXIT_THRESHOLD and jerk_mag < JERK_THRESHOLDS[i]:
                                        state_counters[i] += 1
                                        if state_counters[i] >= SUSTAINED_FRAMES and current_states[i] != "Smooth":
                                            current_states[i] = "Smooth"
                                            state_counters[i] = 0
                                    else:
                                        state_counters[i] = 0

                        previous_accelerations = accelerations
                    previous_velocities = velocities
                previous_time = current_time

                for i, index in enumerate(SELECTED_LANDMARKS):
                    landmark = smoothed_landmarks[i]
                    x, y = int(landmark[0] * frame.shape[1]), int(landmark[1] * frame.shape[0])
                    label = current_states[i]
                    color = (255, 255, 255) if label == "Still" else (0, 255, 0) if label == "Smooth" else (0, 0, 255)
                    cv2.circle(image, (x, y), 5, color, -1)
                    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

               
                dominant_state = max(set(current_states), key=current_states.count)
                send_to_max(dominant_state)

            cv2.imshow("Movement Classification", image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    start_detection()
